In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, DoubleType, IntegerType
from pyspark.sql.functions import sum as _sum, avg, expr, window, from_unixtime, col

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, TimestampType
import time, datetime
import os
import random
import uuid

spark = (
    SparkSession.builder.appName("spark application")
    .config(
        "spark.jars.packages",
        "org.apache.hudi:hudi-spark3.1-bundle_2.12:0.13.0,"
        "org.apache.spark:spark-avro_2.12:3.1.1,"
        "org.apache.hadoop:hadoop-aws:3.1.1,"
        "com.amazonaws:aws-java-sdk:1.11.271,"
    )
    .config(
        "spark.sql.extensions",
        "org.apache.spark.sql.hudi.HoodieSparkSessionExtension",
    )
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .config(
        "spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem"
    )
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config(
        "spark.hadoop.fs.s3a.aws.credentials.provider",
        "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider",
    )
    .config("spark.hadoop.fs.s3a.connection.maximum", "1000")
    .getOrCreate()
)


23/06/19 16:19:57 WARN Utils: Your hostname, ducdn resolves to a loopback address: 127.0.1.1; using 10.1.124.58 instead (on interface enp1s0)
23/06/19 16:19:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/ducdn/.ivy2/cache
The jars for the packages stored in: /home/ducdn/.ivy2/jars
org.apache.hudi#hudi-spark3.1-bundle_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4d9962a0-b853-4584-a087-894515cf29d2;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/ducdn/Documents/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.hudi#hudi-spark3.1-bundle_2.12;0.13.0 in central
	found org.apache.spark#spark-avro_2.12;3.1.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-aws;3.1.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.271 in central
	found com.amazonaws#aws-java-sdk;1.11.271 in central
	found com.amazonaws#aws-java-sdk-transcribe;1.11.271 in central
	found com.amazonaws#aws-java-sdk-core;1.11.271 in central
	found commons-logging#commons-logging;1.1.3 in central
	found org.apache.httpcomponents#httpclient;4.5.2 in central
	found org.apache.httpcomponents#httpcore;4.4.4 in central
	found commons-codec#commons-codec;1.9 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found com.fasterxml.jackson.core#jackson-databind;2.6.7.1 in central
	found com.fasterxml.jackson.core#jackson-annotations;2.6.0 in central
	found com.fasterxml.jackson.core#jackson-core;2.6.7 in central
	found com.fasterxml.jackson.dataformat#jacks

In [4]:

hoodie_options = {
    "hoodie.table.name": "hudi_mor",
    "hoodie.metadata.enable": "true",
    "hoodie.table.type": "MERGE_ON_READ",
    "hoodie.datasource.write.table.type": "MERGE_ON_READ",
    "hoodie.datasource.write.operation": "upsert",
    "hoodie.datasource.write.recordkey.field": "id",
    "hoodie.datasource.write.partitionpath.field": "partition",
    "hoodie.datasource.write.table.name": "hudi_mor",
    "hoodie.datasource.write.precombine.field": "timestamp",
    "hoodie.clean.automatic": "true",
    "hoodie.cleaner.policy": "KEEP_LATEST_FILE_VERSIONS",
    "hoodie.cleaner.fileversions.retained": 10,
    "hoodie.compact.inline": "true",
    "hoodie.compact.inline.max.delta.commits": 3,
    "hoodie.datasource.write.hive_style_partitioning": "true",
    "hoodie.datasource.hive_sync.enable": "true",
    "hoodie.datasource.hive_sync.mode": "hms",
    "hoodie.datasource.hive_sync.database": "default",
    "hoodie.datasource.hive_sync.table": f"hudi_mor",
    "hoodie.datasource.hive_sync.partition_fields": "partition",
    "hoodie.datasource.hive_sync.partition_extractor_class": 
        "org.apache.hudi.hive.MultiPartKeysValueExtractor",
    "hoodie.datasource.hive_sync.metastore.uris": 
        "thrift://hive-metastore:9083"
}

In [5]:
schema = StructType([
    StructField("id", StringType()),
    StructField("value", StringType()),
    StructField("timestamp", IntegerType()),
    StructField("partition", StringType())
])

#### Update

In [7]:

data = [("cee11856-3a7d-499e-a9b4-98a875fa04ad", 11223344, current_timestamp(), 1)]
stream_df = spark.createDataFrame(data, schema)


stream_df.write.format("hudi") \
    .options(**hoodie_options) \
    .mode("append") \
    .save(f"s3a://datalake/hudi_mor")


23/06/19 16:55:25 WARN HoodieWriteConfig: Embedded timeline server is disabled, fallback to use direct marker type for spark
23/06/19 16:55:27 WARN HoodieWriteConfig: Embedded timeline server is disabled, fallback to use direct marker type for spark
23/06/19 16:55:32 WARN HoodieWriteConfig: Embedded timeline server is disabled, fallback to use direct marker type for spark


#### Write Hudi Mor

In [8]:
def current_timestamp():
    current_time = int(time.time())
    return current_time

def gen_data():
    data = []
    for i in range(0, 5):
        i = uuid.uuid4()
        value = random.randint(0, 1000000)
        timestamp = current_timestamp()
        partition = random.randint(1,3)
        x = (str(i), value, timestamp, partition)
        data.append(x)
    return data

while True:
    print("="* 20 + "START GEN DATA" + "="*20)
    data = gen_data()
    stream_df = spark.createDataFrame(data, schema)
    # stream_df = stream_df.withColumn("timestamp", current_timestamp())

    stream_df.show(n=2, truncate=False)

    stream_df.write.format("hudi") \
        .options(**hoodie_options) \
        .mode("append") \
        .save(f"s3a://datalake/hudi_mor")

    print(20*"-" + "STARTING WRITE HUDI TABLE" + "-"*20)

    print(20*"-" + "DONE" + "-"*20)

    print(f"================== TIME SLEEP ==============")
    time.sleep(10)



====================START GEN DATA====================
+------------------------------------+------+----------+---------+
|id                                  |value |timestamp |partition|
+------------------------------------+------+----------+---------+
|fd943a8d-218d-415c-a17a-4272144edbc2|738998|1687170177|3        |
|20d8ed05-41a9-4c56-ba1f-cbd742e28b17|28269 |1687170177|3        |
+------------------------------------+------+----------+---------+
only showing top 2 rows



23/06/19 17:22:57 WARN HoodieWriteConfig: Embedded timeline server is disabled, fallback to use direct marker type for spark
23/06/19 17:23:00 WARN HoodieWriteConfig: Embedded timeline server is disabled, fallback to use direct marker type for spark


--------------------STARTING WRITE HUDI TABLE--------------------
--------------------DONE--------------------
================== TIME SLEEP ==============
====================START GEN DATA====================
+------------------------------------+------+----------+---------+
|id                                  |value |timestamp |partition|
+------------------------------------+------+----------+---------+
|4e591885-d71d-4edc-9eb5-7af3ec87cf6d|664253|1687170193|2        |
|69fe012b-8ae6-4db0-8788-c07b46b829ae|56747 |1687170193|2        |
+------------------------------------+------+----------+---------+
only showing top 2 rows



23/06/19 17:23:14 WARN HoodieWriteConfig: Embedded timeline server is disabled, fallback to use direct marker type for spark
23/06/19 17:23:15 WARN HoodieWriteConfig: Embedded timeline server is disabled, fallback to use direct marker type for spark


--------------------STARTING WRITE HUDI TABLE--------------------
--------------------DONE--------------------
================== TIME SLEEP ==============
====================START GEN DATA====================
+------------------------------------+------+----------+---------+
|id                                  |value |timestamp |partition|
+------------------------------------+------+----------+---------+
|75b3084c-c148-4995-8453-ca156babc20c|263073|1687170208|1        |
|f6634bc1-6db0-4767-9384-3da2c58879ba|952050|1687170208|2        |
+------------------------------------+------+----------+---------+
only showing top 2 rows



...

23/06/19 17:27:39 WARN HoodieWriteConfig: Embedded timeline server is disabled, fallback to use direct marker type for spark
23/06/19 17:27:41 WARN HoodieWriteConfig: Embedded timeline server is disabled, fallback to use direct marker type for spark
23/06/19 17:27:44 WARN HoodieWriteConfig: Embedded timeline server is disabled, fallback to use direct marker type for spark
23/06/19 17:27:46 WARN BaseHoodieCompactionPlanGenerator: No operations are retrieved for s3a://datalake/hudi_mor


--------------------STARTING WRITE HUDI TABLE--------------------
--------------------DONE--------------------
================== TIME SLEEP ==============
====================START GEN DATA====================
+------------------------------------+------+----------+---------+
|id                                  |value |timestamp |partition|
+------------------------------------+------+----------+---------+
|20df2193-9f65-4a86-84c4-e6ba353ebb3d|975558|1687170476|2        |
|fd0dd050-9d9f-4ba4-87d8-63866d8bdb72|365408|1687170476|3        |
+------------------------------------+------+----------+---------+
only showing top 2 rows



23/06/19 17:27:57 WARN HoodieWriteConfig: Embedded timeline server is disabled, fallback to use direct marker type for spark
23/06/19 17:27:59 WARN HoodieWriteConfig: Embedded timeline server is disabled, fallback to use direct marker type for spark
23/06/19 17:28:02 WARN HoodieWriteConfig: Embedded timeline server is disabled, fallback to use direct marker type for spark
23/06/19 17:28:05 WARN BaseHoodieCompactionPlanGenerator: No operations are retrieved for s3a://datalake/hudi_mor


--------------------STARTING WRITE HUDI TABLE--------------------
--------------------DONE--------------------
================== TIME SLEEP ==============


KeyboardInterrupt: 